# LSTM Time Series Forecasting Input / Output Shape

###  Univariate(단변수) Multi-step Input LSTM and Single-step Output
이 모델은 단변수 시계열 데이터에 대한 LSTM을 사용하여 여러 타임 스텝의 입력을 기반으로 단일 타임 스텝의 출력을 생성합니다. 단변수 시계열 데이터란 하나의 특성만을 가진 데이터를 의미합니다. 여기서, 모델은 과거의 일정한 시간 윈도우(window) 내 데이터를 사용하여 미래의 한 단계 값을 예측합니다. 예를 들어, 과거 30일 동안의 주식 가격을 기반으로 내일의 주식 가격을 예측할 수 있습니다.

### Multivariate(다변수) Multi-step Input LSTM and Single-step Output
이 모델은 다변수 시계열 데이터에 대한 LSTM을 사용하여 여러 타임 스텝의 입력을 기반으로 단일 타임 스텝의 출력을 생성합니다. 다변수 시계열 데이터란 여러 개의 특성을 가진 데이터를 의미합니다. 이 모델은 단변수 버전과 비슷하지만, 여러 특성을 사용하여 예측을 수행합니다. 예를 들어, 주식 시장에서 과거 30일 동안의 주가, 거래량, 시가총액 등의 정보를 사용하여 내일의 주식 가격을 예측할 수 있습니다.

여기서 **multi-step size**란 **window size**를 의미합니다.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential

## 1. Univariate Multi-step Input and Single-step output LSTM 

- 단일변수 multi-timestep 입력 단일 timestep 출력  

- input feature - 1, output unit - 1

    ex) 과거 3 일간 종가 입력 $\rightarrow$ 내일 주가 예상

In [2]:
def windowed_ds(series, window_size, batch_size, shuffle_buffer):    
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size+1))
    ds = ds.map(lambda window: (window[:-1], window[-1]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds

In [3]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

window_size = 3
batch_size = 1
dataset = windowed_ds(raw_seq, window_size, batch_size, 10)
dataset

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [4]:
for x, y in dataset:
    print(x.numpy().shape, y.numpy().shape)
    print(x.numpy())
    print(y.numpy())
    break

(1, 3) (1,)
[[10 20 30]]
[40]


In [5]:
model = Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=[None, 1]),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


Train이 안된 상태에서 간단히 모델을 작동시켜 입출력의 shape을 확인해 봅니다. 훈련이 안된 상태이므로 당연히 yhat 값은 의미가 없습니다.

In [6]:
x_input = np.array([[70, 80, 90]])
x_input = x_input.reshape(1, -1, 1)
yhat = model.predict(x_input[:window_size])

print(x_input.shape, yhat.shape)
print(x_input, yhat)

1/1 [==============================] - 0s 115ms/step
(1, 3, 1) (1, 1)
[[[70]
  [80]
  [90]]] [[11.35783]]


## 2. Multivariate Multi-step Input and Single-step Output LSTM 

- 여러개의 변수를 multi-timestep 입력 $\rightarrow$ 단일 time-step 출력  

- input feature - n, output unit - 1

    ex) 주가, 환율 과거 3 일치 입력하여 다음날 주가(환율) 예측
    ```
    [[ 10,  15],
     [ 20,  25],
     [ 30,  35]]   --> [40]   
    ```

- input sequence 정의

In [7]:
in_stock = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_forex = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])

out_seq = in_stock[3:]
out_seq

array([40, 50, 60, 70, 80, 90])

- `[row, columns]` 구조로 변환하고 열을 수평으로 쌓습니다.

여기서 in_stock과 in_forex 두 배열을 각각 2차원으로 변환하고, np.hstack() 함수를 사용하여 두 배열을 수평으로 연결합니다. 결과적으로 raw_seq는 두 배열을 나란히 놓은 형태의 2차원 배열이 됩니다. 

In [8]:
in_stock = in_stock.reshape(-1, 1)
in_forex = in_forex.reshape(-1, 1)

raw_seq = np.hstack((in_stock, in_forex))
raw_seq

array([[10, 15],
       [20, 25],
       [30, 35],
       [40, 45],
       [50, 55],
       [60, 65],
       [70, 75],
       [80, 85],
       [90, 95]])

ds.map()을 사용하여 입력 특성과 레이블로 구성된 훈련 샘플을 생성합니다. 여기서 window[:-1]는 마지막 단계를 제외한 모든 단계의 모든 feature를 사용하여 입력 특성을 구성하고, window[-1, 0]는 마지막 단계의 첫 번째 feature(예: 주식 가격)을 레이블로 사용합니다.

In [9]:
def windowed_ds(series, window_size, batch_size, shuffle_buffer):    
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size+1))
    ds = ds.map(lambda window: (window[:-1], window[-1, 0]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds

In [10]:
window_size = 3
batch_size = 1
dataset = windowed_ds(raw_seq, window_size, batch_size, 10)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 2), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [11]:
for x, y in dataset:
    print(x.numpy().shape, y.numpy().shape)
    print(x.numpy(), y.numpy())
    break

(1, 3, 2) (1,)
[[[10 15]
  [20 25]
  [30 35]]] [40]


In [12]:
model = Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=[None, 2]),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10600     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________


In [13]:
x_input = np.array([[70, 80]])
x_input = x_input.reshape(1, -1, 2)
yhat = model.predict(x_input[:window_size])

print(x_input.shape, yhat.shape)

1/1 [==============================] - 0s 113ms/step
(1, 1, 2) (1, 1)


### 다변수를 이용한 주가 예측

In [14]:
import yfinance as yf

df = yf.download('AAPL', start='2015-01-01', end='2019-12-31', progress=False)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.565704,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.873640,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.875891,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.210680,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.140907,237458000


In [15]:
dataset = df.iloc[:, [3, 5]].values
dataset.shape

(1257, 2)

In [16]:
window_size = 3
batch_size = 1

ds = tf.expand_dims(dataset, axis=1)
ds = tf.data.Dataset.from_tensor_slices(dataset)
ds = ds.window(window_size+1, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(window_size+1))
ds = ds.map(lambda w: (w[:-1], w[-1][0]))
ds = ds.batch(batch_size).prefetch(1)
ds

for x, y in ds:
    print(x.shape, y.shape)
    print()
    print(x.numpy(), y.numpy())
    break

(1, 3, 2) (1,)

[[[2.73325005e+01 2.12818400e+08]
  [2.65625000e+01 2.57142000e+08]
  [2.65650005e+01 2.63188400e+08]]] [26.9375]
